# size distribution example
Load basic packages

In [4]:
import statistics

import pandas as pd
import pandas_bokeh
import numpy as np
from pylab import *
from scipy.optimize import curve_fit
import scipy
import csv

from bokeh.io import output_file, output_notebook
from bokeh.models import DatetimeTickFormatter, ranges, Span, Label, Range1d, LinearAxis
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.palettes import linear_palette, Turbo256
from matplotlib import pyplot as plt
pd.set_option('plotting.backend', 'pandas_bokeh')
output_notebook(hide_banner=True)
from IPython.core.display import display, HTML



In [5]:
def get_X_List( values ):
    value_cnt = len(values)
    x_range = list(range(value_cnt))
    x_range.pop(0)
    x_range.append(value_cnt)
    return x_range

def get_x(x, c):
    c_len = len(c)
    if c_len == 1:
        return c[0]
    else:
        return get_x(x, c[:-1])*x + c[c_len-1]

def get_polyfit_xy( x, y, deg=3, resolution=1 ):
    c = np.polynomial.polynomial.polyfit(x, y, deg)

    x_result = np.linspace(x[0], x[-1], num=len(x)*10)
    y_result = np.polynomial.polynomial.polyval(x_result, c)
    return x_result,y_result

def extrapolation_function(x, a, b, c):
    return a * np.exp(-b * x) + c

# see https://en.wikipedia.org/wiki/Inverse_Gaussian_distribution
def inv_gaussian_dist(x, lamb, mu, c=1.0):
    # since this causes problems with the curve_fit function and x is always larger than 0 in this case, the following two lines are not needed
    #if x <= 0.0:
    #    return 0.0              
    return c * np.sqrt( lamb / ( 2.0 * np.pi * x**3 )) * np.exp( ( -lamb * (x - mu)**2 ) / ( 2 * mu**2 * x ) )

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fatiguelife.html#scipy.stats.fatiguelife
def fatiguelife(x, c):
    return scipy.stats.fatiguelife.pdf(x, c)#, loc, scale

def pareto(x,c):
    return scipy.stats.pareto.pdf(x, c)#, loc, scale

def powerlognorm(x, c, s):
    return scipy.stats.powerlognorm.pdf(x, c, s)

def get_bin_center(bins):
    bin_center = []
    for l, r in zip(bins[1:], bins[:-1]):
        bin_center.append((r-l)/2+l)
    return bin_center

def get_histogram(dist_type='cld', x_column='diameter', y_column='count', max_value=None, plot_width=1200, legend_location='top_right', smoothing=0, do_fit=False, show_sum_list=False):
    # some basic label settings
    if dist_type == 'psd':
    #    df = psd.psd_df
        dist_name = 'pore size distribution'
        x_min_f=1
    else:
    #    df = psd.cld_df
        dist_name = 'chord length distribution'
    
    unit = 'nm³'
    power = 1


    # get & process the actual histogram


    # max_value has to be given in the same unit as in self.scaling['unit']
    voids = pd.read_csv( './Voids.Label-Analysis.csv' )
    scaling = 1.61236*1.61236*10
    #print(scaling)
    max_value = 100000# 500*500*50/500
    bin_count = round( max_value/scaling+2 )
    #print(bin_count)
    bins = np.array( [round(i*scaling,4) for i in range(0, bin_count, 1)] )

    bin_center = get_bin_center(bins)
    histogram = np.histogram(list(voids['Volume3d']), bins=bins)
    hist = histogram[0]#*bin_center
    
    x_min = bins[0]
    
    if max_value == None: max_value = bins[-1]
    x_max = bins[-1] if max_value >= bins[-1] else max_value
    
    title = 'histogram of pore volume'#'{} as {} of {} cld_complete_histogram_median, x_column, psd.filename, max_value, unit'
    x_label = 'pore colume in {}'.format(unit)
    y_label = 'pore count per bin'

    if x_column=='area':
        x_axis_type='log'
    elif x_column=='volume':
        x_axis_type='log'
    elif x_column=='surface':
        x_axis_type='log'
    else:
        x_axis_type='linear'

    # get the center of the bins

    # normalize the histograms
    normalized_hist = hist#/(np.sum(histogram[0]*bin_center)/100)
    y_max = np.amax(normalized_hist)
    
    first_val_pos = 0
    
    # create the figure
    p = figure(title=title,  plot_width=plot_width, x_range=(x_min, x_max), y_range=(0,y_max), plot_height=600, background_fill_color="#fafafa", x_axis_type=x_axis_type)
    p.legend.location = legend_location
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label

    # histogram of raw data
    p.quad(bottom=0, top=hist, left=bins[1:], right=bins[:-1], fill_color="blue", line_color="blue")

    # smoothed raw data
    if False:#smoothing > 2:
        hist_smooth = sdc.smooth_histogram(normalized_hist, window_len=smoothing)
        p.line(bin_center, hist_smooth, line_color="#ff88ff", line_width=4, alpha=0.7, legend_label="smoothed")

    #polynome fit
    if False:#do_fit:
        # fit to a predefined function
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html  lamb, mu, c
        
        first_val_y = bin_center[first_val_pos]
        popt, pcov = curve_fit(powerlognorm, bin_center[first_val_pos:], normalized_hist[first_val_pos:], bounds=([0, -np.inf], np.inf))
        
        label='-'
        x_fit = np.linspace(int(bin_center[first_val_pos:][0]), int(bin_center[-1])+1, num=len(bin_center)*2)
        y_fit = powerlognorm(x_fit-first_val_y, *popt)
        p.line(x_fit, y_fit, line_color="#11aa11", line_width=2, alpha=0.7, legend_label=label)

    if False:#show_sum_list:
        # iterated sum
        sum_list = psd.get_sum_list(normalized_hist)
        p.extra_y_ranges = {"sum_line": Range1d(start=0, end=np.amax(sum_list))}

        p.line(bins, sum_list, line_color="#ff8888", line_width=4, alpha=0.7, legend_label="Sum", y_range_name="sum_line")
        p.add_layout(LinearAxis(y_range_name="sum_line",axis_label=y_label), 'left')

    
    show(p)

generate Diagrams 

In [6]:
get_histogram('cld','diameter', show_sum_list=False)